In [67]:
import d2lzh as d2l
import math
from mxnet import autograd, nd
from mxnet.gluon import loss as gloss
import time

In [68]:
corpus_indices, char_to_idx, idx_to_char, vocab_size = d2l.load_data_jay_lyrics()

In [69]:
nd.one_hot(nd.array([0, 2]), vocab_size)


[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]
<NDArray 2x1027 @cpu(0)>

In [70]:
def to_onehot(X, size):
    return [nd.one_hot(x, size) for x in X.T]

X = nd.arange(10).reshape((2, 5))
inputs = to_onehot(X, vocab_size)
len(inputs), inputs[0].shape

(5, (2, 1027))

In [71]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
ctx = d2l.try_gpu()
print('will use', ctx)

def get_params():
    def _one(shape):
        return nd.random.normal(scale=0.01, shape=shape, ctx=ctx)
    
    W_xh = _one((num_inputs, num_hiddens))
    W_hh = _one((num_hiddens, num_hiddens))
    b_h = nd.zeros(num_hiddens, ctx=ctx)
    
    W_hq = _one((num_hiddens, num_outputs))
    b_q = nd.zeros(num_outputs, ctx=ctx)
    
    params = [W_xh, W_hh, b_h, W_hq, b_q]
    for param in params:
        param.attach_grad()
    return params

will use cpu(0)


In [72]:
def init_rnn_state(batch_size, num_hiddens, ctx):
    return (nd.zeros(shape=(batch_size, num_hiddens), ctx=ctx))

def rnn(inputs, state, params):
    W_xh, W_hh, b_h, W_hq, b_q = params
    H = state
    outputs = []
    for X in inputs:
        H = nd.tanh(nd.dot(X, W_xh) + nd.dot(H, W_hh) + b_h)
        Y = nd.dot(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, H

In [73]:
state = init_rnn_state(X.shape[0], num_hiddens, ctx)
inputs = to_onehot(X.as_in_context(ctx), vocab_size)
params = get_params()
outputs, state_new = rnn(inputs, state, params)
len(outputs), outputs[0].shape, state_new[0].shape

(5, (2, 1027), (256,))

In [74]:
def predict_rnn(prefix, num_chars, rnn, params, init_rnn_state,
               num_hiddens, vocab_size, ctx, idx_to_char, char_to_idx):
    state = init_rnn_state(1, num_hiddens, ctx)
    output = [char_to_idx[prefix[0]]]
    for t in range(num_chars + len(prefix) -  1):
        X = to_onehot(nd.array([output[-1]], ctx=ctx), vocab_size)
        (Y, state) = rnn(X, state, params)
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(int(Y[0].argmax(axis=1).asscalar()))
    return ''.join([idx_to_char[i] for i in output])

In [75]:
predict_rnn('分开', 10, rnn, params, init_rnn_state, num_hiddens, vocab_size,
           ctx, idx_to_char, char_to_idx)

'分开离拜梦撑酸飞熬步坏兮'

In [76]:
def grad_clipping(params, theta, ctx):
    norm = nd.array([0], ctx)
    for param  in params:
        norm += (param.grad ** 2).sum()
    norm = norm.sqrt().asscalar()
    if norm > theta:
        for param in params:
            param.grad[:] *= theta / norm

In [77]:
def train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens, 
                          vocab_size, ctx, corpus_indices, idx_to_char,
                         char_to_idx, is_random_iter, num_epochs, num_steps,
                         lr, clipping_theta, batch_size, pred_period,
                         pred_len, prefixes):
    if is_random_iter:
        data_iter_fn = d2l.data_iter_random
    else:
        data_iter_fn = d2l.data_iter_consecutive
    params = get_params()
    loss = gloss.SoftmaxCrossEntropyLoss()
    
    for epoch in range(num_epochs):
        if not is_random_iter:
            state = init_rnn_state(batch_size, num_hiddens, ctx)
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_fn(corpus_indices, batch_size, num_steps, ctx)
        for X, Y in data_iter:
            if is_random_iter:
                state = init_rnn_state(batch_size, num_hiddens, ctx)
            else:
                for s in  state:
                    s.detach()
            with autograd.record():
                inputs = to_onehot(X, vocab_size)
                (outputs, state) = rnn(inputs, state, params)
                outputs = nd.concat(*outputs, dim=0)
                y = Y.T.reshape((-1, ))
                l = loss(outputs, y).mean()
            l.backward()
            grad_clipping(params, clipping_theta, ctx)
            d2l.sgd(params, lr, 1)
            l_sum += l.asscalar() * y.size
            n += y.size
          
        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch +1, math.exp(l_sum / n), time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn(
                prefix, pred_len, rnn, params, init_rnn_state,
                num_hiddens, vocab_size, ctx, idx_to_char, char_to_idx))

In [80]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 250, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                     vocab_size, ctx, corpus_indices, idx_to_char,
                     char_to_idx, True, num_epochs, num_steps, lr,
                     clipping_theta, batch_size, pred_period, pred_len,
                     prefixes)

epoch 50, perplexity 68.794496, time 1.76 sec
 - 分开 我不要再 你你的让 在谁的可  我不能 我怎么这 我有我有 你你的让 在人的可  我在你 我不了这
 - 不分开  我有你 我不了这 我有我有 你你的让 在人的可  我在你 我不了这 我有我有 你你的让 在人的可
epoch 100, perplexity 10.112449, time 1.35 sec
 - 分开 一只令 一步两步三颗四步 连成线背著背 我该想好生你的世 就时线背著背 我该道好生 我不能再想 我
 - 不分开 我给道好生瘦 世知的觉 你知经 别人堂 三着我 说你都 旧什么 一颗四 干什么 干什么 干什么 干
epoch 150, perplexity 2.914917, time 1.43 sec
 - 分开 一只令它心 我唱懂拖抽 黑不在美落 有蝪什么走 不里在美落 不里在停留 不里在美留 不里在美落 不
 - 不分开  我不能我想 我不要再想 我不 我不 我不要再想你 不知不觉 你已经离开我 不知不觉 我跟了这节奏
epoch 200, perplexity 1.599260, time 1.46 sec
 - 分开 有愿段到我 谁子寄停豆喝能分力的 脑海当呜前迹止啦 不要再这样打我妈妈 难道你手不会痛吗 不要再这
 - 不分开吗 我叫你爸 你打我妈 这样对吗干嘛这样 不要再酒牵打斯走 瞎 说我心仪睡我 想物就这样打我妈攻  
epoch 250, perplexity 1.330184, time 1.85 sec
 - 分开 出只主人心 谁钩它 一句惹毛我的人有危险 一再重演 一根我不抽的菸 一点个   用单爱 说不布为说
 - 不分开期把单单没有伤害 你 靠着我的肩膀 你 在我胸口睡著 像这样的生活 我爱你 你爱我 开不了口 周杰伦
